## Preprocess data into 15 intervals

## Imports

In [1]:
import pandas as pd
from os.path import join
import datetime

%matplotlib inline
pd.set_option("display.precision", 2)

In [2]:
PATH = join("data_preprocessed", "historia_przejazdow_2019-07.csv")
PATH

'data_preprocessed/historia_przejazdow_2019-07.csv'

In [3]:
df = pd.read_csv(PATH, index_col=0, parse_dates=True)

df.head()

,start_time,end_time,rental_place,return_place
bike_number,,,,
57254,2019-06-30 23:54:42,2019-07-01 00:00:44,42,90
57413,2019-06-30 23:18:03,2019-07-01 00:01:01,86,61
650703,2019-06-30 23:51:05,2019-07-01 00:01:02,196,154
650374,2019-06-30 23:50:54,2019-07-01 00:01:24,112,115
650264,2019-06-30 23:57:33,2019-07-01 00:01:24,189,178


In [4]:
df['start_time']= pd.to_datetime(df['start_time']) 
df['end_time']= pd.to_datetime(df['end_time'])
df.head()

,start_time,end_time,rental_place,return_place
bike_number,,,,
57254,2019-06-30 23:54:42,2019-07-01 00:00:44,42,90
57413,2019-06-30 23:18:03,2019-07-01 00:01:01,86,61
650703,2019-06-30 23:51:05,2019-07-01 00:01:02,196,154
650374,2019-06-30 23:50:54,2019-07-01 00:01:24,112,115
650264,2019-06-30 23:57:33,2019-07-01 00:01:24,189,178


## Divide in intervals

### Earliest rental 

In [5]:
start = df.start_time.min()
start = start.replace(hour=0, minute=0, second=0)
start

Timestamp('2019-06-30 00:00:00')

### Latest return

In [6]:
end = df.end_time.max()
end = end.replace(hour=0, minute=0, second=0) + datetime.timedelta(days=1)

end

Timestamp('2019-08-01 00:00:00')

In [7]:
ranges = pd.date_range(start, end,freq='15T')
ranges

DatetimeIndex(['2019-06-30 00:00:00', '2019-06-30 00:15:00',
               '2019-06-30 00:30:00', '2019-06-30 00:45:00',
               '2019-06-30 01:00:00', '2019-06-30 01:15:00',
               '2019-06-30 01:30:00', '2019-06-30 01:45:00',
               '2019-06-30 02:00:00', '2019-06-30 02:15:00',
               ...
               '2019-07-31 21:45:00', '2019-07-31 22:00:00',
               '2019-07-31 22:15:00', '2019-07-31 22:30:00',
               '2019-07-31 22:45:00', '2019-07-31 23:00:00',
               '2019-07-31 23:15:00', '2019-07-31 23:30:00',
               '2019-07-31 23:45:00', '2019-08-01 00:00:00'],
              dtype='datetime64[ns]', length=3073, freq='15T')

In [8]:
all_trips_in_all_intervals_df  = pd.DataFrame(columns=["interval_start", "interval_end", "rental_place", "return_place"])
all_trips_in_all_intervals_df   

,interval_start,interval_end,rental_place,return_place


In [9]:
for i in range(len(ranges)-1):
    interval_start, interval_end = ranges[i],ranges[i+1]
    
    interval_df = df[(df.start_time <= interval_end) & (df.end_time >= interval_start)]    
    
    
    interval_df["interval_start"] = interval_start
    interval_df["interval_end"] = interval_end
    
    all_trips_in_all_intervals_df = all_trips_in_all_intervals_df.append(interval_df[["interval_start", "interval_end", "rental_place", "return_place"]])

<ipython-input-9-a70ccd99ed5b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interval_df["interval_start"] = interval_start
<ipython-input-9-a70ccd99ed5b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interval_df["interval_end"] = interval_end


In [10]:
all_trips_in_all_intervals_df

,interval_start,interval_end,rental_place,return_place
57369,2019-06-30 15:30:00,2019-06-30 15:45:00,97,154
57369,2019-06-30 15:45:00,2019-06-30 16:00:00,97,154
57369,2019-06-30 16:00:00,2019-06-30 16:15:00,97,154
57369,2019-06-30 16:15:00,2019-06-30 16:30:00,97,154
57369,2019-06-30 16:30:00,2019-06-30 16:45:00,97,154
...,...,...,...,...
650444,2019-07-31 23:45:00,2019-08-01 00:00:00,198,111
650780,2019-07-31 23:45:00,2019-08-01 00:00:00,121,119
650003,2019-07-31 23:45:00,2019-08-01 00:00:00,92,83
650520,2019-07-31 23:45:00,2019-08-01 00:00:00,128,38


## Group by rentsl/return place

In [11]:
intervals_grouped_df = pd.DataFrame(columns=["interval_start", "interval_end", "number_of_trips", "rental_place", "return_place"])

intervals_grouped_df

,interval_start,interval_end,number_of_trips,rental_place,return_place


In [12]:
for i in range(len(ranges)-1):
    interval_start, interval_end = ranges[i],ranges[i+1]
    
    interval_group_df = (all_trips_in_all_intervals_df[all_trips_in_all_intervals_df["interval_start"] == interval_start]).groupby(['rental_place', 'return_place'], as_index=False).count()

    interval_group_df = interval_group_df.rename(columns={"interval_start": "number_of_trips"})

    interval_group_df["interval_start"] = interval_start
    interval_group_df["interval_end"] = interval_end
    
    intervals_grouped_df = intervals_grouped_df.append(interval_group_df)

In [13]:
intervals_grouped_df

,interval_start,interval_end,number_of_trips,rental_place,return_place
0,2019-06-30 15:30:00,2019-06-30 15:45:00,1,97,154
0,2019-06-30 15:45:00,2019-06-30 16:00:00,1,97,154
0,2019-06-30 16:00:00,2019-06-30 16:15:00,1,97,154
0,2019-06-30 16:15:00,2019-06-30 16:30:00,1,97,154
0,2019-06-30 16:30:00,2019-06-30 16:45:00,1,97,154
...,...,...,...,...,...
37,2019-07-31 23:45:00,2019-08-01 00:00:00,1,161,62
38,2019-07-31 23:45:00,2019-08-01 00:00:00,1,172,61
39,2019-07-31 23:45:00,2019-08-01 00:00:00,1,190,176
40,2019-07-31 23:45:00,2019-08-01 00:00:00,1,196,196


## Save to CSV

In [24]:
intervals_grouped_df.to_csv(join("plik.csv"), index=False)

In [22]:
intervals_grouped_df.groupby(["interval_start", "interval_end"], as_index=False).count()["number_of_trips"].max()

533